Concept Clustering

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dmp.data.pmlb import pmlb_loader
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import sys
sys.path.append("..")

from networks import get_basic_537_houses
from viz import *

In [3]:
model, inputs, outputs = get_basic_537_houses(num_hidden_layers=6, hidden_layer_width=128)

(20640, 8)
array([[0.90226638],
       [0.70824656],
       [0.69505074],
       ...,
       [0.15938285],
       [0.14371281],
       [0.15340349]])
(20640, 1)
num_observations 20640 num_inputs 8 num_outputs 1
Epoch 1/100
65/65 [==============================] - 2s 17ms/step - loss: 0.0454 - val_loss: 0.0190
Epoch 2/100
65/65 [==============================] - 0s 3ms/step - loss: 0.0196 - val_loss: 0.0206
Epoch 3/100
65/65 [==============================] - 0s 3ms/step - loss: 0.0181 - val_loss: 0.0182
Epoch 4/100
65/65 [==============================] - 0s 3ms/step - loss: 0.0175 - val_loss: 0.0172
Epoch 5/100
65/65 [==============================] - 0s 3ms/step - loss: 0.0165 - val_loss: 0.0172
Epoch 6/100
65/65 [==============================] - 0s 3ms/step - loss: 0.0153 - val_loss: 0.0168
Epoch 7/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0155 - val_loss: 0.0184
Epoch 8/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0147 - val_loss: 0.0

In [12]:
import numpy

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [40]:
def _get_forward_activations(model:tf.keras.Sequential, X_data:numpy.ndarray):
    """
    Utility function to compute activations of each unit in model with respect to X_data
    """

    keras_inputs = [model.input]
    keras_outputs = [layer.output for layer in model.layers]
    activations = tf.keras.backend.function(keras_inputs, keras_outputs)([X_data])

    return activations

def layerwise_concept_clustering(model:tf.keras.Sequential, inputs, outputs, artifact_path=False, dimensionalty_reduction="pca", clustering="kmeans"):
    activations = _get_forward_activations(model, inputs)

    reduced_activations = []
    clusters = []
    n_cluster_max = []

    for activation in activations:
        pca = PCA(2)
        pca.fit(activation)
        reduced_activation = pca.transform(activation)

        scores = []
        for n_clusters in range(2,5):
            print(activation.shape[0])
            print(n_clusters)
            clusterer = KMeans(n_clusters=n_clusters)
            cluster_labels = clusterer.fit_predict(reduced_activation)
            print(cluster_labels)
            scores.append( silhouette_score(reduced_activation, cluster_labels) )

        n_cluster_max.append( numpy.argmax(scores) )



    return n_cluster_max

In [41]:
n_clusters = layerwise_concept_clustering(model, inputs, outputs)

20640
2
[0 0 0 ... 0 0 0]
20640
3
[1 1 1 ... 1 1 1]


KeyboardInterrupt: 

In [39]:
n_clusters

[0, 0, 1, 0, 0, 1, 0]